# Load libraries

In [9]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import sklearn
import imblearn
import pickle
from joblib import load, dump

from sklearn.metrics import f1_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier

In [10]:
# Download features
# !scp -r -P 22334 -i ~/.ssh/id_rsa guanaco.inf.uach.cl:/home/shared/astro/PLAsTiCC/fats_featurs.tar.gz .
# !tar xzvf fats_features.tar.gz

# Get data

In [11]:
# Select seed, load ids and features

seed = 0
with open(f"ids/seed{seed}/maxClass15k/dataset_ids_before_balancing.pkl", "rb") as f:
    lc_ids = pickle.load(f)

# lc_ids["train"].
features = {}
for subset in ['train', 'validation', 'test']:
    print(subset)
    tmp = []
    
#     tmp2 = lc_ids[subset][:10]
    
#     for lc_id in tmp2:
    for lc_id in lc_ids[subset]:
#         print(lc_id)
#         print("aoisdj")
#         lc_id = lc_id[:2]
        with open("features/fats"+str(int(lc_id))+".pkl", "rb") as f:
            tmp.append(load(f))
    features[subset] = pd.concat(tmp, axis=0)

train
validation
test


In [18]:
print(features["test"].fillna(-1000).values.shape)

print(lc_ids['labels_test'].astype('int').shape)

(6546, 288)
(6546,)


# Train RF model

In [51]:
# Train balanced RF
# from sklearn.metrics import f1_score, classification_report
# from imblearn.ensemble import BalancedRandomForestClassifier

rf = BalancedRandomForestClassifier(n_estimators=500, criterion='entropy', replacement=True,
                                    max_depth=10, class_weight='balanced', n_jobs=8) 
                

rf.fit(
    features['train'].fillna(-1000).values, 
    lc_ids['labels_train'].astype('int'),
) 

# save RF trained model
dump(rf, '../../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib') 
print("RF model saved")

# preds = rf.predict(features['validation'].fillna(-1000).values)
# print(classification_report(preds, lc_ids['labels_validation'].astype('int')))
# # print(rf.features)          

RF model saved


# Load saved RF model

In [52]:
# # load file
rf = load('../../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib') 

# Get test model predictions

In [43]:
# assert lc_ids["test"].shape == lc_ids["labels_test"].shape == preds.shape

In [54]:
# Result on test set
preds = rf.predict(features['test'].fillna(-1000).values)

# print(preds.shape)

# save results
results = {
    "testIds": lc_ids["test"],
    "testLabels": lc_ids["labels_test"],
    "testPredictions": preds,
}

# save object
a_file = open("../../experiments/comparingModels/seed" + str(seed) + "/RF/testPredictionsRF.pkl", "wb")
pickle.dump(results, a_file)
a_file.close()

# # print(preds)
# print(classification_report(preds, lc_ids['labels_test'][:10].astype('int')))

In [41]:
# load files
# a_file = open("../../experiments/comparingModels/" + "/testPredictionsRF.pkl", "rb")
# output = pickle.load(a_file)
# print(output["testPredictions"])

In [8]:
# # Features sorted by importance
# features['train'].columns.values[np.argsort(rf.feature_importances_)[::-1]]